In [ ]:
# Importación de Librerías
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split, KFold

from sklearn.metrics import cohen_kappa_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv', index_col="PetID")
train.info()

In [ ]:
for c in train.select_dtypes("O"):
    train[c] = train[c].astype("category")

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
def metric(y_true, y_pred):
    res = cohen_kappa_score(y_true, y_pred.reshape((y_true.shape[0], 5), order="F").argmax(axis=1), weights= 'quadratic')
    return "kappa", res, True

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train.select_dtypes(exclude=['object']).drop("AdoptionSpeed", axis=1), 
    train.AdoptionSpeed, random_state=1
)
kf = KFold(n_splits=5, random_state=1, shuffle=True)

In [ ]:
folds = [X_train.index[idx] for _, idx in kf.split(X_train)]

In [ ]:
valid_probs = []
test_probs = pd.DataFrame([], index=X_test.index, columns=range(y_train.max() + 1))
for idx in folds:
    Xv = X_train.loc[idx]
    yv = y_train.loc[idx]
    
    Xt = X_train.drop(idx)
    yt = y_train.drop(idx)
    
    model = LGBMClassifier()
    model.fit(Xt, yt)
    
    vp = pd.Series(model.predict_proba(Xv), index=Xv)
    valid_probs.append(vp)
    
    test_probs = test_probs + pd.Series(model.predict_proba(X_test), index=X_test.index) / 5
valid_probs = pd.concat(valid_probs)